In [1]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from nlp import load_dataset
import nlp
import torch
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
model = BertForSequenceClassification.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
RANDOM_SEED = 50
MAX_LEN = 512
EPOCHS=4
if torch.cuda.device_count() > 1:
    BATCH_SIZE = torch.cuda.device_count() * 8
else:
    BATCH_SIZE = 8

In [4]:
dataset = load_dataset('csv', data_files='./reviews.csv', split='train')
shuffled_ds = dataset.shuffle(RANDOM_SEED)
split_ds = shuffled_ds.train_test_split(test_size=0.2)
train_dataset = split_ds['train']
test_val_dataset = split_ds['test']
split_tv = test_val_dataset.train_test_split(test_size=0.5)
test_dataset = split_tv['train']
val_dataset = split_tv['test']


Using custom data configuration default


In [5]:
def sample(ds, ratio):
    return random.sample(range(0, len(ds)), int(len(ds) * ratio)) 
'''
ds_names = ['train', 'test', 'val']
for name in ds_names:
    ds_name = eval("{}_dataset".format(name))
    ds_name = ds_name.select(indices=sample(ds_name, 0.1))
    # train_dataset.select(indices=sample(train_dataset, 0.1))
'''
train_dataset = train_dataset.select(indices=sample(train_dataset, 0.1))
test_dataset = test_dataset.select(indices=sample(test_dataset, 0.1))
val_dataset = val_dataset.select(indices=sample(val_dataset, 0.1))


In [6]:
def tokenize(batch):
    return tokenizer(batch['content'], max_length=MAX_LEN, padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [7]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [8]:
trainer.train()

/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.
/home/joshhu/anaconda3/envs/nlp/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


TrainOutput(global_step=13668, training_loss=0.17246795553844319)

In [9]:
trainer.evaluate()

{'eval_loss': 0.16734198627413296,
 'eval_accuracy': 0.9616510538641686,
 'eval_f1': 0.9669108360697146,
 'eval_precision': 0.967644084934277,
 'eval_recall': 0.9661786976274609,
 'epoch': 4.0}

In [10]:
%load_ext tensorboard
%tensorboard --logdir './logs'